We provide all products and services under the terms of the Creative Commons Attribution-ShareAlike 4.0 International licence (CC BY-SA 4.0).

The data and database are open and licensed by the Open Data Commons Open Database License (ODbL).

Under these licences, you can freely use our products and data for non-commercial or commercial purposes and credit OpenWeather as a weather data source in a visible part of your application.

For an Enterprise account, we can work under customised licences. Please contact us to discuss your requirements and conditions.

In [5]:
import requests
import datetime as dt
from geopy.geocoders import Nominatim
import os
import numpy as np
import pandas as pd


In [32]:
def OpenWeatherMap_scraper(location_name=None, longitude=None, latitude=None, units='metric'):
    #import packages
    import requests
    import datetime as dt
    import time
    import geopy
    from geopy.geocoders import Nominatim
    import numpy as np
    import pandas as pd

    #determine latitiude and longitude based on what was passed to function (may change from what was )
    if location_name:
        locator = Nominatim(user_agent='myGeocoder')
        location = locator.geocode(location_name)
        lat = location.latitude
        lon = location.longitude
    else: 
        lat = str(latitude)
        lon = str(longitiude)
    
    #datetime = dt.datetime.strptime(date, '%m/%d/%Y')
    datetimeunix = time.mktime(dt.date.today().timetuple())
    
    exclude='current,minutely,hourly'
    
    #define API key
    API_key = '617dec18b0669db53a5ab299c4e48609'
    
    #One Call: Current weather, Minute forecast for 1 hour, Hourly forecast for 48 hours, Daily forecast for 7 days
    #National weather alerts, Historical weather data for the previous 5 days
    url = r'''https://api.openweathermap.org/data/2.5/onecall?lat={lat}&lon={lon}&exclude={exclude}&units={units}&appid={API_key}'''.format(lat=lat, lon=lon, API_key=API_key, exclude=exclude, time=datetimeunix, units=units)

    response = requests.get(url)
    return response.json()

In [33]:
scraped = OpenWeatherMap_scraper(location_name='boston, ma')
scraped

{'lat': 42.3603,
 'lon': -71.0583,
 'timezone': 'America/New_York',
 'timezone_offset': -14400,
 'daily': [{'dt': 1621958400,
   'sunrise': 1621934049,
   'sunset': 1621987715,
   'moonrise': 1621985100,
   'moonset': 1621931940,
   'moon_phase': 0.47,
   'temp': {'day': 22.13,
    'min': 10.34,
    'max': 24.72,
    'night': 17.08,
    'eve': 23.51,
    'morn': 10.59},
   'feels_like': {'day': 21.57, 'night': 16.7, 'eve': 23.22, 'morn': 9.45},
   'pressure': 1022,
   'humidity': 45,
   'dew_point': 9.67,
   'wind_speed': 6.46,
   'wind_deg': 222,
   'wind_gust': 13.72,
   'weather': [{'id': 803,
     'main': 'Clouds',
     'description': 'broken clouds',
     'icon': '04d'}],
   'clouds': 75,
   'pop': 0,
   'uvi': 9.04},
  {'dt': 1622044800,
   'sunrise': 1622020408,
   'sunset': 1622074169,
   'moonrise': 1622076360,
   'moonset': 1622020620,
   'moon_phase': 0.5,
   'temp': {'day': 29.5,
    'min': 16.3,
    'max': 31.47,
    'night': 19.24,
    'eve': 27.85,
    'morn': 17.14},
  

In [34]:
daily=pd.DataFrame(scraped['daily'])

In [35]:
daily_drop = ['sunrise','sunset','moonrise','moonset','moon_phase','feels_like',
              'pressure','humidity','dew_point', 'wind_speed','wind_deg','wind_gust',
              'clouds','uvi', 'id', 'icon']

In [52]:
scraped['daily'][0]['weather'][0]

{'id': 803, 'main': 'Clouds', 'description': 'broken clouds', 'icon': '04d'}

In [47]:
daily_1 = pd.concat([daily.drop(['temp'], axis=1), daily['temp'].apply(lambda x: pd.Series(x[0]))], axis=1)
daily_1

KeyError: 0

In [42]:
daily_1.weather

0    [{'id': 803, 'main': 'Clouds', 'description': ...
1    [{'id': 501, 'main': 'Rain', 'description': 'm...
2    [{'id': 501, 'main': 'Rain', 'description': 'm...
3    [{'id': 501, 'main': 'Rain', 'description': 'm...
4    [{'id': 501, 'main': 'Rain', 'description': 'm...
5    [{'id': 501, 'main': 'Rain', 'description': 'm...
6    [{'id': 500, 'main': 'Rain', 'description': 'l...
7    [{'id': 500, 'main': 'Rain', 'description': 'l...
Name: weather, dtype: object

In [43]:
daily_1 = pd.concat([daily_1.drop(['weather'], axis=1), daily_1['weather'].apply(pd.Series)], axis=1)

In [44]:
daily_1#.drop(columns=daily_drop)

,dt,sunrise,sunset,moonrise,moonset,moon_phase,feels_like,pressure,humidity,dew_point,...,pop,uvi,rain,day,min,max,night,eve,morn,0
0,1621958400,1621934049,1621987715,1621985100,1621931940,0.47,"{'day': 21.57, 'night': 16.7, 'eve': 23.22, 'm...",1022,45,9.67,...,0.00,9.04,NaN,22.13,10.34,24.72,17.08,23.51,10.59,"{'id': 803, 'main': 'Clouds', 'description': '..."
1,1622044800,1622020408,1622074169,1622076360,1622020620,0.50,"{'day': 29.81, 'night': 19.75, 'eve': 28.73, '...",1013,46,16.45,...,1.00,9.52,5.34,29.50,16.30,31.47,19.24,27.85,17.14,"{'id': 501, 'main': 'Rain', 'description': 'mo..."
2,1622131200,1622106769,1622160622,1622167320,1622109780,0.55,"{'day': 22.85, 'night': 14.59, 'eve': 21.28, '...",1013,40,8.79,...,1.00,8.59,1.36,23.41,15.69,24.71,15.69,21.98,17.96,"{'id': 501, 'main': 'Rain', 'description': 'mo..."
3,1622217600,1622193132,1622247074,1622257680,1622199540,0.59,"{'day': 15.25, 'night': 7.98, 'eve': 12.93, 'm...",1019,43,3.47,...,1.00,6.40,9.90,16.43,9.49,16.51,9.49,14.08,11.75,"{'id': 501, 'main': 'Rain', 'description': 'mo..."
4,1622304000,1622279497,1622333525,0,1622289780,0.62,"{'day': 6.16, 'night': 5.72, 'eve': 5.78, 'mor...",1018,94,8.23,...,1.00,5.74,25.94,9.26,9.03,9.55,9.19,9.18,9.55,"{'id': 501, 'main': 'Rain', 'description': 'mo..."
5,1622390400,1622365864,1622419975,1622347320,1622380440,0.66,"{'day': 6.38, 'night': 6.69, 'eve': 6.76, 'mor...",1021,95,8.63,...,1.00,0.42,17.97,9.61,9.00,9.80,9.80,9.68,9.27,"{'id': 501, 'main': 'Rain', 'description': 'mo..."
6,1622476800,1622452233,1622506423,1622436240,1622471100,0.69,"{'day': 9.97, 'night': 9.84, 'eve': 12.95, 'mo...",1016,96,9.54,...,1.00,1.00,4.05,10.37,9.05,13.51,10.66,13.51,10.16,"{'id': 500, 'main': 'Rain', 'description': 'li..."
7,1622563200,1622538604,1622592870,1622524680,1622561700,0.73,"{'day': 19, 'night': 15.64, 'eve': 21.33, 'mor...",1021,51,9.05,...,0.29,1.00,0.10,19.65,8.68,21.77,15.93,21.77,8.68,"{'id': 500, 'main': 'Rain', 'description': 'li..."


In [10]:
weather = pd.DataFrame(scraped['current']['weather'])
scraped['current'].pop('weather')

current = pd.DataFrame(scraped['current'], index=range(len(scraped['current'])))
scraped.pop('current')

alerts = pd.DataFrame(scraped['alerts'])
scraped.pop('alerts')
scraped

{'lat': 42.3603,
 'lon': -71.0583,
 'timezone': 'America/New_York',
 'timezone_offset': -14400}

In [12]:
latlon = pd.DataFrame(scraped, index=range(len(scraped)))

In [23]:
latlon

,lat,lon,timezone,timezone_offset
0,42.3603,-71.0583,America/New_York,-14400
1,42.3603,-71.0583,America/New_York,-14400
2,42.3603,-71.0583,America/New_York,-14400
3,42.3603,-71.0583,America/New_York,-14400


In [22]:
alerts[['event','start','end']]

,sender_name,event,start,end,description,tags
0,NWS Boston (Southeast New England),Small Craft Advisory,1621974000,1621998000,...SMALL CRAFT ADVISORY REMAINS IN EFFECT UNTI...,[]


In [20]:
current[['dt','temp']]

,dt,temp
0,1621996629,17.4
1,1621996629,17.4
2,1621996629,17.4
3,1621996629,17.4
4,1621996629,17.4
5,1621996629,17.4
6,1621996629,17.4
7,1621996629,17.4
8,1621996629,17.4
9,1621996629,17.4


In [17]:
weather[['main', 'description']]

,main,description
0,Clouds,broken clouds


In [ ]:
output_clean = output.drop(columns=['feels_like', 'pressure', 'humidity', 'dew_point', 'uvi', 'clouds'])

In [25]:
response.json()

{'lat': 41.2273,
 'lon': -73.0667,
 'timezone': 'America/New_York',
 'timezone_offset': -14400,
 'current': {'dt': 1621965626,
  'sunrise': 1621934738,
  'sunset': 1621987990,
  'temp': 294.26,
  'feels_like': 294.02,
  'pressure': 1024,
  'humidity': 61,
  'dew_point': 286.45,
  'uvi': 8.81,
  'clouds': 88,
  'visibility': 10000,
  'wind_speed': 2.24,
  'wind_deg': 205,
  'wind_gust': 8.05,
  'weather': [{'id': 804,
    'main': 'Clouds',
    'description': 'overcast clouds',
    'icon': '04d'}]},
 'minutely': [{'dt': 1621965660, 'precipitation': 0},
  {'dt': 1621965720, 'precipitation': 0},
  {'dt': 1621965780, 'precipitation': 0},
  {'dt': 1621965840, 'precipitation': 0},
  {'dt': 1621965900, 'precipitation': 0},
  {'dt': 1621965960, 'precipitation': 0},
  {'dt': 1621966020, 'precipitation': 0},
  {'dt': 1621966080, 'precipitation': 0},
  {'dt': 1621966140, 'precipitation': 0},
  {'dt': 1621966200, 'precipitation': 0},
  {'dt': 1621966260, 'precipitation': 0},
  {'dt': 1621966320, 'p

In [ ]:
daily = pd.DataFrame(scraped['daily'])
daily['dt'] = pd.to_datetime(daily['dt'], unit='s', utc=True)
daily['dt']

In [4]:
locator = Nominatim(user_agent='myGeocoder')
coor = '41.227250, -73.066658'
location = locator.reverse(coor)
location

Location(98, Davarlin Terrace, Milford, New Haven County, Connecticut, 06460, United States, (41.226734, -73.066054, 0.0))

In [ ]:
date = '5/24/2021'
import time

In [ ]:
datetime = dt.datetime.strptime(date, '%m/%d/%Y')
datetimeunix = time.mktime(datetime.timetuple())
datetimeunix

In [ ]:
datetime.fromtimestamp(datetimeunix)